## Define the Environment
First, we'll create an environment class that simulates the SIS epidemic dynamics, including the arrival and departure processes.

In [1]:
import numpy as np

class SISEpidemicEnv:
    def __init__(self, n_susceptible, n_infected, lambda_arr, lambda_dep, infection_rate, recovery_rate):
        self.initial_state = (n_susceptible, n_infected, lambda_arr, lambda_dep)
        self.infection_rate = infection_rate
        self.recovery_rate = recovery_rate
        self.reset()

    def reset(self):
        self.state = self.initial_state
        return self.state

    def step(self, action):
        n_susceptible, n_infected, lambda_arr, lambda_dep = self.state
        
        # Simulate arrivals and departures
        arrivals = np.random.poisson(lambda_arr)
        departures = np.random.poisson(lambda_dep)
        
        # Update susceptible and infected counts
        new_infected = np.random.binomial(n_susceptible, 1 - np.exp(-self.infection_rate * n_infected))
        new_recovered = np.random.binomial(n_infected, 1 - np.exp(-self.recovery_rate))
        
        n_susceptible = max(n_susceptible + arrivals - departures - new_infected, 0)
        n_infected = max(n_infected + new_infected - new_recovered, 0)
        
        # Update state
        self.state = (n_susceptible, n_infected, lambda_arr, lambda_dep)
        
        # Calculate reward (negative of the number of infected individuals)
        reward = -n_infected
        
        return self.state, reward


Constructor (__init__): Initializes the environment with the initial number of susceptible (n_susceptible) and infected (n_infected) individuals, arrival (lambda_arr) and departure (lambda_dep) rates, infection rate (infection_rate), and recovery rate (recovery_rate).
Reset: Resets the environment to the initial state and returns it.
Step: Simulates one time step in the environment. This involves:
Simulating arrivals and departures using Poisson processes.
Updating the number of susceptible and infected individuals based on the infection and recovery processes.
Returning the new state and the reward (negative of the number of infected individuals).


## Implement Q-Learning Algorithm
Next, we'll implement the Q-learning algorithm to learn the optimal policy.

In [2]:


class QLearningAgent:
    def __init__(self, state_bins, action_space, learning_rate=0.1, discount_factor=0.99, exploration_rate=1.0, exploration_decay=0.995):
        self.state_bins = state_bins
        self.action_space = action_space
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.exploration_decay = exploration_decay
        self.q_table = np.zeros(tuple(len(bins) + 1 for bins in state_bins) + (action_space,))
        
    def choose_action(self, state):
        if np.random.rand() < self.exploration_rate:
            return np.random.choice(self.action_space)
        return np.argmin(self.q_table[state])
    
    def learn(self, state, action, reward, next_state):
        best_next_action = np.argmin(self.q_table[next_state])
        td_target = reward + self.discount_factor * self.q_table[next_state + (best_next_action,)]
        td_error = td_target - self.q_table[state + (action,)]
        self.q_table[state + (action,)] += self.learning_rate * td_error
        self.exploration_rate *= self.exploration_decay
        
def discretize_state(state, state_bins):
    return tuple(np.digitize(s, bins) - 1 for s, bins in zip(state, state_bins))

# Define state and action space bins
n_susceptible_bins = np.linspace(0, 100, 10)
n_infected_bins = np.linspace(0, 50, 10)
lambda_arr_bins = np.linspace(0, 0, 5)
lambda_dep_bins = np.linspace(0, 0, 5)
state_bins = [n_susceptible_bins, n_infected_bins, lambda_arr_bins, lambda_dep_bins]

action_space = 3  # Example: 0: No action, 1: Quarantine, 2: Vaccination

# Initialize environment and agent
env = SISEpidemicEnv(100, 10, 2, 1, 0.05, 0.01)
agent = QLearningAgent(state_bins, action_space)

# Training
n_episodes = 1000

for episode in range(n_episodes):
    state = env.reset()
    state = discretize_state(state, state_bins)
    
    done = False
    while not done:
        action = agent.choose_action(state)
        next_state, reward = env.step(action)
        next_state = discretize_state(next_state, state_bins)
        
        agent.learn(state, action, reward, next_state)
        
        state = next_state
        if reward == 0:  # termination condition: no infected individuals? Is this the reason why it is non  convergent?
            done = True # new terimnaitoni condition for suboptimality, less than 5 or 3% of the population is infected (termaiton at time before )


print("Training completed.")


KeyboardInterrupt: 

Constructor (__init__): Initializes the Q-learning agent with state bins, action space, learning rate, discount factor, exploration rate, and exploration decay. It also initializes the Q-table.
Choose Action: Chooses an action based on an epsilon-greedy strategy (random action with probability exploration_rate, otherwise the action with the minimum Q-value).
Learn: Updates the Q-table using the Q-learning update rule:
Calculates the TD target and TD error.
Updates the Q-value for the current state-action pair.
Decays the exploration rate.


## Evaluate the Learned Policy

Finally, we evaluate the learned policy by running the environment with the learned Q-values.

In [ ]:

# Evaluation
state = env.reset()
state = discretize_state(state, state_bins)
done = False
total_reward = 0

while not done:
    action = np.argmin(agent.q_table[state])
    next_state, reward = env.step(action)
    next_state = discretize_state(next_state, state_bins)
    
    total_reward += reward
    state = next_state
    if reward == 0:  # Example termination condition: no infected individuals
        done = True

print("Evaluation completed. Total reward:", total_reward)


IndexError: index 10 is out of bounds for axis 0 with size 10

In [ ]:
import matplotlib.pyplot as plt

# Plot the results
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(rewards_per_episode)
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.title('Total Reward per Episode')

# Plot optimal control policy for different initial states
optimal_actions = np.zeros((len(n_susceptible_bins), len(n_infected_bins)))

for i, n_susceptible in enumerate(n_susceptible_bins):
    for j, n_infected in enumerate(n_infected_bins):
        state = (n_susceptible, n_infected, 2, 1)  # Fixing lambda_arr and lambda_dep for visualization
        discretized_state = discretize_state(state, state_bins)
        optimal_action = np.argmin(agent.q_table[discretized_state])
        optimal_actions[i, j] = optimal_action

plt.subplot(1, 2, 2)
plt.imshow(optimal_actions, cmap='viridis', origin='lower')
plt.colorbar(ticks=[0, 1, 2])
plt.xticks(range(len(n_infected_bins)), np.round(n_infected_bins, 2), rotation=90)
plt.yticks(range(len(n_susceptible_bins)), np.round(n_susceptible_bins, 2))
plt.xlabel('Number of Infected')
plt.ylabel('Number of Susceptible')
plt.title('Optimal Actions')

plt.tight_layout()
plt.show()

# Evaluation
state = env.reset()
state = discretize_state(state, state_bins)
done = False
total_reward = 0
state_trajectory = [state]

while not done:
    action = np.argmin(agent.q_table[state])
    next_state, reward = env.step(action)
    next_state = discretize_state(next_state, state_bins)
    
    total_reward += reward
    state = next_state
    state_trajectory.append(state)
    if reward == 0:  # Example termination condition: no infected individuals
        done = True

print("Evaluation completed. Total reward:", total_reward)

# Plot state trajectory during evaluation
state_trajectory = np.array(state_trajectory)
plt.figure()
plt.plot(state_trajectory[:, 0], label='Susceptible')
plt.plot(state_trajectory[:, 1], label='Infected')
plt.xlabel('Time step')
plt.ylabel('Count')
plt.title('State Trajectory during Evaluation')
plt.legend()
plt.show()
